In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2:latest",
    temperature=0.7
)

In [8]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human","請分析這家餐廳:{restaurant_name}")
    ]
)

In [11]:
# 定義優點分析函數
def analyze_pros(restaurant_info):
    """分析餐廳的優點"""
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的優點。"),
            ("human","根據餐廳資訊：{restaurant_info}，請列出這家餐廳的優點。")
        ]
    )

    return pros_template.format_prompt(restaurant_info=restaurant_info)

def analyze_cons(restaurant_info):
    """分析餐廳的優點"""
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的缺點。"),
            ("human","根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。")
        ]
    )

    return cons_template.format_prompt(restaurant_info=restaurant_info)

print("分析函數已定義：")
print("1. analyze_pros: 分析餐廳優點")
print("2. analyze_cons: 分析餐廳缺點")

分析函數已定義：
1. analyze_pros: 分析餐廳優點
2. analyze_cons: 分析餐廳缺點


In [12]:
# 定義結果合併函數
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

print("結果合併函數已定義：combine_pros_cons")


結果合併函數已定義：combine_pros_cons


In [13]:
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

In [14]:
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(
        pros=pros_branch_chain,
        cons=cons_branch_chain
    )
    | RunnableLambda(lambda x: combine_pros_cons(x["pros"], x["cons"]))
)

In [15]:
# 執行並行處理鏈 - 測試餐廳評論分析系統
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""
result = chain.invoke({"restaurant_name": restaurant_info})

# 輸出結果
print("=" * 60)
print("餐廳評論分析結果：")
print("=" * 60)
print(result)

餐廳評論分析結果：

🍽️ 餐廳評論分析報告

✅ 優點分析：
這家義大利餐廳的優點包括：

1. 環境優雅：餐廳內間設計得很精美，氛圍溫馨和舒適，是一個適合 romantic dinner 的選擇。
2. 服務人員親切：伺服人員都是熱情和親切的，讓客人感到舒適和放鬆。
3. 傳統義式料理：餐廳提供了正宗的義大利菜，對於愛好這類料理的食客來說是一個很好的選擇。

❌ 缺點分析：
這家義大利餐廳的缺點有：

1.價格偏高
2. 等待時間較長
3. 份量偏少

📊 總結：這是一個平衡的餐廳評論分析

